<img src="pr1.png" />

In [1]:
import cvxpy
import gurobipy
from cvxpy import *
import numpy as np

In [24]:
np.random.seed(42)

# initial matrix
m = np.array([[ 5 , 5 , 10, 10, 10 ],
              [ 5 , 5 , 10, 20, 10 ],
              [ 0 , 5 , 5 , 10,  5 ],
              [ 0 , 0 , 0 , 5 ,  0 ]])

N, M = m.shape
C = N * M

# delta matrix
d = np.ones((N, M)) * 6
d = d - m

def cost(i1, j1, i2, j2):
    assert(i1 < N and i2 < N and j1 < M and j2 < M and 
           i1 >= 0 and i2 >= 0 and j1 >= 0 and j2 >= 0)
    
    if i1 == i2 and j1 == j2:
        return 1000000
    
    return np.sqrt(np.power(np.abs(i1 - i2), 2) + np.power(np.abs(j1 - j2), 2))

m = m.reshape(C,1)
d = d.reshape(C,1)

f = Int(C * C)
c = np.zeros(C * C)

for i1 in xrange(N):
    for j1 in xrange(M):
        for i2 in xrange(N):
            for j2 in xrange(M):
                c[(i1 * M + j1) * C + (i2 * M + j2)] = cost(i1, j1, i2, j2)
                
# each cell can either be only a consumer or "donor".
# otherwise we might get non integer solution
def pred(x):
    if x < 0:
        return 1
    
    return 0

flows_are_positive = [ (f[i] >= 0) for i in xrange(C * C) ]

pd = np.vectorize(pred)
is_donor = pd(d).reshape(C,)
flow_network_constraints = [ (-is_donor[i] * cvxpy.sum_entries(f[i*C:i*C + C]) + \
                             (1 - is_donor[i]) * cvxpy.sum_entries(f[i:C*C:C]) == d[i]) \
                            for i in xrange(C) ]

fnc_not_donors = [ (1 - is_donor[i]) * cvxpy.sum_entries(f[i*C:i*C + C]) == 0 for i in xrange(C) ]
fnc_donors = [ is_donor[i] * cvxpy.sum_entries(f[i:C*C:C]) == 0 for i in xrange(C) ]

obj = Minimize(c * f)

constraints = flows_are_positive + flow_network_constraints + fnc_not_donors + fnc_donors

#print installed_solvers()

prob = Problem(obj, constraints)
res = prob.solve(feastol=1e-8) # verbose=True, 
print "Result: ", res
eps = 0.00001
print "Flows size: ", f.value[np.where( f.value > eps )] 
# print cvxpy.sum_entries(f[13:C*C:C]).value

Result:  95.4910638371
Here are the flows size:  [[ 1.  1.  1.  1.  4.  1.  3.  2.  1.  1.  1.  5.  6.  2.  1.  3.  4.]]


I've tried several similar min cost max flow models. All of them gave the same result as this one that is constrained to have an integer valued flows.

<img src="pr2.png" />

In [12]:
# initialization

import numpy as np
import scipy as sp
from numpy import random
from numpy import matlib
import matplotlib.pyplot as plt

%matplotlib inline

np.random.seed(1)

C = 40; # number of clients
clients = np.random.rand(2,C); #client positions
F = 20; #number of facilities
facilities = np.random.rand(2,F);

#maximum number of clients per facility
capacities = np.array([2, 3, 4, 2, 5, 3, 2, 1, 2, 3, 4, 3, 2, 1, 2, 2, 2, 3, 2, 3]);

dx = np.matlib.repmat(clients[0,:],F,1) - np.matlib.repmat(facilities[0,:],C,1).transpose();
dy = np.matlib.repmat(clients[1,:],F,1) - np.matlib.repmat(facilities[1,:],C,1).transpose();

assignment_costs = 3*(dx*dx + dy*dy); #the assignment cost is the distance squared

#opening_costs = np.ones((F,));
opening_costs = np.array([1, 2, 1, 2, 1, 1.2, 1.3, 2, 1, 1, 1.4, 2, 1, 1, 1, 1, 1.5, 1.3, 2, 1]);

def plot_solution(facilities_opened, clients_assignment):
    plt.scatter(clients[0,:], clients[1,:], s=50.0, c=clients_assignment)
    plt.scatter(facilities[0,:], facilities[1,:], s=200.0, c=range(F), linewidth = [5*el for el in facilities_opened])
    plt.show()

In [ ]:
def lp_solve():
    

def branch_and_bound():
    incumbent_value = 1e10
    
    queue = []

<img src="pr3.png" />